
##FMP API calls and DFs

###Justin Parsons  
###MSDS692 S8W1 2025

This notebook pulls current and historical employee counts as well as ESG data from FMP. It uses a separate library written for doing API calls to Financial Modeling Prep, "FMP_API_library" which is available within the GHG groups repo.

*There are multiple hardcoded environment-dependent lines within this script, but they are easily identified and changed without breaking the code.*

---


**Change Log**

If I was better I'd do some sort of version control...

1.   V1.0 JTP 2/1/2025  Initial version
2.   



In [ ]:
import sys
from google.colab import drive

#link google drive and set path. Only necessary if using Google drive to store code and data.

drive.mount('/content/drive/')


#hardcoded for Justins' environment
sys.path.append('/content/drive/MyDrive/GHG/')
sys.path.append('/content/drive/MyDrive/GHG/Code/')
sys.path.append('/content/drive/MyDrive/GHG/Data/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#libraries
import pandas as pd
import requests
import datetime
from time import sleep
import certifi
from urllib.request import urlopen
import json
import time
import re
import ssl

#Library file co-contained with the notebooks. Be aware that path is correct when using a different environment
from FMP_API_library import *

In [ ]:
###
# User defined parameters and environment specifics
###
#year = 2013 #edit year as appropriate NOT APPLICABLE FOR ALL requests may require uncommenting within FMP_API_library

input_path = "/content/drive/MyDrive/GHG/Data/"
output_path = "/content/drive/MyDrive/GHG/Data/"

#API key removed. FMP API is paid for by GHG research group.
API_Key = "######################"     #API key provided for class 300 requests per minute limit



mm_df is the "Mega Merged" file which contains the GHG group's data. It is read into a dataframe here to get identifiers (Symbol_1) for the API calls. Eventually this will be userdefined or an separate file of unique symbols will be created.

In [ ]:
#reading in to get Symbol_1 as ticker within the functions
mm_all_df = pd.read_csv(input_path + "Mega_merged_all_real_values.csv")
mm_df.info()

<ipython-input-7-77155e61f4cd>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  mm_all_df = pd.read_csv(input_path + "Mega_merged_all_real_values.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181156 entries, 0 to 181155
Data columns (total 42 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   account_id                    181156 non-null  int64  
 1   account_name                  181156 non-null  object 
 2   incorporated_country          181156 non-null  object 
 3   ticker                        117989 non-null  object 
 4   Evaluation_status             180307 non-null  object 
 5   Scope_3_emissions_type        181156 non-null  object 
 6   Scope_3_emissions_amount      181156 non-null  float64
 7   Symbol_1                      181156 non-null  object 
 8   Year                          181156 non-null  int64  
 9   Market_Cap                    115901 non-null  float64
 10  Employee_Count                23397 non-null   float64
 11  ebitda                        118470 non-null  float64
 12  Revenue                       118470 non-nul

Below all of the API functions are called and data is read into separate dataframes prior to being merged with the Mega Merged dataframe in a separate .

In [ ]:
%%time
#combining all API function calls but producing independent outputs
request_counter = 0
start_time = time.time()

# Define the file name with the year variable
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
ESG_file_name = f'ESG_data_{timestamp}.csv'
EC_file_name = f'EC_data_{timestamp}.csv'
EC_hist_file_name = f'EC_hist_data_{timestamp}.csv'

# Combine the directory path and file name
ESG_output_file_path_and_name = output_path + ESG_file_name
EC_output_file_path_and_name = output_path + EC_file_name
EC_hist_output_file_path_and_name = output_path + EC_hist_file_name

#initialize DF
ESG_df = pd.DataFrame()
EC_df = pd.DataFrame()
EC_hist_df = pd.DataFrame()

###
# Calling API functions
###
ESG_df = get_ESG_V2(mm_df, API_Key)
EC_df = get_employeeCount(mm_df, API_Key)
EC_hist_df = get_hist_employeeCount(mm_df, API_Key)

# Save the DataFrame to the specified file path
ESG_df.to_csv(ESG_output_file_path_and_name, index=False)
EC_df.to_csv(EC_output_file_path_and_name, index=False)
EC_hist_df.to_csv(EC_hist_output_file_path_and_name, index=False)